##0.Librerías necesarias

In [12]:
#!pip install rarfile
#!pip install unzip
!pip install stop-words
!pip install num2words


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [54]:
#Librerias necesarias
from google.colab import drive
import numpy as np
import pandas as pd
from google.colab import drive
import random
#import rarfile # para descomprimir archivos
import json
import csv 
#para reducir la dimensión
from sklearn.manifold import TSNE
#para representar
import matplotlib.pyplot as plt
#preprocesado
import unicodedata
import re
import nltk
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk import RegexpTokenizer
from nltk.stem import WordNetLemmatizer 
nltk.download('stopwords')
from nltk.corpus import stopwords
from stop_words import get_stop_words
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
nltk.download('omw-1.4')
import string
from num2words import num2words
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [14]:
# Montamos GDrive para almacenar los datos
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##1. Carga de datos

In [15]:
#cargamos los datos necesarios desde el gDrive
path = '/content/drive/MyDrive/NLP/'

train = pd.read_csv(f'{path}train.csv')
test= pd.read_csv(f'{path}test.csv')

In [16]:
train

,review_text,reviews_sentiment,words,sentiment_label
0,When the pie arrived it was missing one of the...,2,"['When', 'the', 'pie', 'arrived', 'it', 'was',...",1
1,Burger was fabulous. Bacon tasted like one fro...,5,"['Burger', 'was', 'fabulous', '.', 'Bacon', 't...",0
2,"Some of the food was very good - chicken, spam...",3,"['Some', 'of', 'the', 'food', 'was', 'very', '...",1
3,This is a solid taco place. But their tortilla...,4,"['This', 'is', 'solid', 'taco', 'place', '.', ...",1
4,"I had the mahi mahi tacos, which were pretty g...",4,"['I', 'had', 'the', 'mahi', 'mahi', 'tacos', '...",1
...,...,...,...,...
4760,"Its an Ok burger for the price. The Fries, usu...",4,"['Its', 'an', 'Ok', 'burger', 'for', 'the', 'p...",1
4761,I ordered the Cheesy Todd cheeseburger which h...,3,"['I', 'ordered', 'the', 'Cheesy', 'Todd', 'che...",1
4762,"The coffee is surprisingly good, omeletes are ...",3,"['The', 'coffee', 'is', 'surprisingly', 'good'...",1
4763,It's no secret they've got the best burgers ar...,5,"['It', ""'s"", 'secret', 'they', ""'ve"", 'got', '...",0


In [17]:
test

,review_text,reviews_sentiment,words,sentiment_label
0,Next time I want to order bibim naengmyeon and...,5,"['Next', 'time', 'I', 'want', 'to', 'order', '...",0
1,"Love this place, exelent steaks, chumichurri s...",5,"['Love', 'this', 'place', ',', 'exelent', 'ste...",0
2,My server suggested salt & vinegar seasoning f...,3,"['My', 'server', 'suggested', 'salt', '&', 'vi...",1
3,"The green beans were shriveled and, quite hone...",1,"['The', 'green', 'beans', 'were', 'shriveled',...",1
4,"So... when one puts up a sign saying ""chicken ...",2,"['So', '...', 'when', 'one', 'puts', 'up', 'si...",1
...,...,...,...,...
1584,"If you love seafood this is the place for you,...",5,"['If', 'you', 'love', 'seafood', 'this', 'is',...",0
1585,"For starters I had the fritto miso, which is t...",5,"['For', 'starters', 'I', 'had', 'the', 'fritto...",0
1586,A&W has always had smooth root beer. I had for...,5,"['A', '&', 'W', 'always', 'had', 'smooth', 'ro...",0
1587,"The tilapia is very crispy and tastes okay, bu...",3,"['The', 'tilapia', 'is', 'very', 'crispy', 'an...",1


## 2. Preprocesado
#### Se realizan los siguientes pasos:

> 1.   Normalizar
2.   Eliminar símbolos y caracteres especiales
3.   Eliminar signos de puntuación
4.   Lematizar
5.   Eliminar stopwords




In [18]:
#creamos una función que nos haga toda la limpieza

stop_words = set(stopwords.words("english")) #cargamos las stop.words
lemmatizer = WordNetLemmatizer() #cargamos el lematizador

def clean_review(review):
    # Convertir a minúsculas
    review = review.lower()
    
    # Eliminar acentos, etc
    text = unicodedata.normalize('NFKD', review).encode('ascii', 'ignore').decode('utf-8', 'ignore')

    # Eliminar los signos de puntuación y caracteres especiales de cada palabra
    token = text.translate(str.maketrans('', '', string.punctuation))

    # Tokenizar las palabras
    words = word_tokenize(token)

    # Lematizar las palabras
    words = [lemmatizer.lemmatize(w) for w in words]

    # Eliminar las stopwords
    words = [w for w in words if not w in stop_words]

    # Unir las palabras en una sola cadena
    clean_review = ' '.join(words)

    return clean_review


####2.1 Preprocesamos el conjunto de train

In [19]:
# Aplicar la función a la columna 'review'
train['review_clean'] = train['review_text'].apply(clean_review)
train
#creamos un df con solo la columna review_clean y sentiment_label
train_clean =train[['review_clean', 'sentiment_label']]
train_clean

,review_clean,sentiment_label
0,pie arrived wa missing one ordered topping pay...,1
1,burger wa fabulous bacon tasted like one green...,0
2,food wa good chicken spam meat,1
3,solid taco place tortilla bit soft,1
4,mahi mahi taco pretty good landlocked tucson c...,1
...,...,...
4760,ok burger price fry usually dont like kind dry...,1
4761,ordered cheesy todd cheeseburger ha bacon jala...,1
4762,coffee surprisingly good omeletes well made an...,1
4763,secret theyve got best burger around,0


####2.2 Preprocesamos el conjunto de test

In [20]:
# Aplicar la función a la columna 'review' y añadirla al df para tener todo ahí
test['review_clean'] = test['review_text'].apply(clean_review)

#creamos un df con solo la columna review_clean y sentiment_label
test_clean =test[['review_clean', 'sentiment_label']]
test_clean

,review_clean,sentiment_label
0,next time want order bibim naengmyeon dumpling...,0
1,love place exelent steak chumichurri steak per...,0
2,server suggested salt vinegar seasoning tater ...,1
3,green bean shriveled quite honestly worst gree...,1
4,one put sign saying chicken fried nationwell k...,1
...,...,...
1584,love seafood place sits right water front lot ...,0
1585,starter fritto miso fried calamari zucchini sh...,0
1586,aw ha always smooth root beer forgotten sweet ...,0
1587,tilapia crispy taste okay onion ring little bl...,1


In [21]:
#observamos el cambio en train
print('Review original: {}'.format(train['review_text'].values[0]))
print('Review procesada: {}'.format(train['review_clean'].values[0]))

Review original: When the pie arrived it was missing one of the ordered toppings. If I pay $34 for 1 pie I prefer it to be correct.
Review procesada: pie arrived wa missing one ordered topping pay 34 1 pie prefer correct


In [22]:
#observamos el cambio en test
print('Review original: {}'.format(test['review_text'].values[0]))
print('Review procesada: {}'.format(test['review_clean'].values[0]))

Review original: Next time I want to order bibim naengmyeon and dumplings and try another one. And the boss and the staff were kind and it was really nice. The dumplings and bibim naengmyeon are delicious. Bibim naengmyeon was very delicious among places I've ever eaten. Mulnaengmyeon is normal. They give me the broth, but it's really delicious, so I want to go back to the broth.
Review procesada: next time want order bibim naengmyeon dumpling try another one bos staff kind wa really nice dumpling bibim naengmyeon delicious bibim naengmyeon wa delicious among place ive ever eaten mulnaengmyeon normal give broth really delicious want go back broth


In [23]:
# guardamos los dfs en nuestro \content
train_clean.to_csv('train_clean.csv', encoding='utf-8', index=False)
test_clean.to_csv('test_clean.csv', encoding='utf-8', index=False)


In [24]:
#compruebo que los he guardado en mi disco de colab
!ls -lah   

total 640K
drwxr-xr-x 1 root root 4.0K Mar  8 10:55 .
drwxr-xr-x 1 root root 4.0K Mar  8 09:20 ..
drwxr-xr-x 4 root root 4.0K Mar  6 17:51 .config
drwx------ 5 root root 4.0K Mar  8 09:23 drive
drwxr-xr-x 1 root root 4.0K Mar  6 17:52 sample_data
-rw-r--r-- 1 root root 154K Mar  8 10:55 test_clean.csv
-rw-r--r-- 1 root root 462K Mar  8 10:55 train_clean.csv


In [25]:
# lo movemos a nuestro en nuestro google drive
!cp /content/train_clean.csv /content/drive/MyDrive/NLP/train_clean.csv
!cp /content/test_clean.csv /content/drive/MyDrive/NLP/test_clean.csv